In [43]:
from modelzipper.tutils import *

data_dir = "/data/zecheng/Long-form-reasoning-data/data/generated_tasks_permutation/IF-qa3"
all_files = auto_read_dir(data_dir)

content = auto_read_data(os.path.join(data_dir, all_files[0]))

number of files with prefix '' and suffix '': 2
begin to read data from /data/zecheng/Long-form-reasoning-data/data/generated_tasks_permutation/IF-qa3/16k.json | file size: 65.62 MB | file type: json


In [40]:
content.keys()

dict_keys(['task', 'length', 'with_IF', 'dataset'])

In [41]:
len(content['dataset'])

1000

In [44]:
content['dataset'][3]['input'] == content['dataset'][0]['input']

False

In [45]:
content['dataset'][3]

{'input': 'Sandra went to the garden. "Yes," replied Dagobert, making a sign of intelligence to the orphans,\n"we have lots to tell you for a fortnight to come; and among other\nthings, how we chanced to meet with Gabriel. All I can now say is that,\nin his way, he is quite as good as my boy (I shall never be tired of\nsaying\'my boy\'); and they ought to love each other like brothers. Oh, my\nbrave, brave wife!" said Dagobert, with emotion, "you did a good thing,\npoor as you were, taking the unfortunate child--and bringing him up with\nyour own." "Don\'t talk so much about it, my dear; it was such a simple thing." "You are right; but I\'ll make you amends for it by and by. \'Tis down to\nyour account; in the mean time, you will be sure to see him to-morrow\nmorning." cried the blacksmith; "who\'ll say, after\nthis, that there are not days set apart for happiness? How came you to\nmeet him, father?" "I\'ll tell you all, by and by, about when and how we met Gabriel; for if\nyou expect 